In [1]:
import pandas as pd
import xlsxwriter
from datetime import datetime
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


In [100]:
# function for Views Amount transformation

def value_to_int(x):
    if type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return int(float(x.replace('K', '')) * 1000)
        return 1000
    if 'M' in x:
        if len(x) > 1:
            return int(float(x.replace('M', '')) * 1000000)
        return 1000000
    else: return x

In [102]:

driver = webdriver.Firefox()
driver2 = webdriver.Firefox()
colName = ['imdb_url', 'season', 'episode_number_relative', 'title', 'airdate', 'rating', 'reviews_amount']

csvfile = "/Users/juliatsymbal/Documents/Bicocca Master/Semester 1/IMDd_Scraper/SimpsonsR.csv"
with open(csvfile, "a") as f:
    writer = csv.writer(f, delimiter=',' )
    writer.writerow(colName)

for season in range(1, 35):

    simps_link = f'https://www.imdb.com/title/tt0096697/episodes?season={season}'
    driver.get(simps_link)

    ep_url_list = driver.find_elements(By.XPATH, "//div[@itemprop='episodes']/strong/a")
    ep_href_list = []
    ep_name_text_list = []
    for item in ep_url_list:
        ep_href_list.append(item.get_attribute("href"))

    # ep_name_list = driver.find_elements(By.XPATH, "//div[@itemprop='episodes']/strong/a")
    # for item in ep_name_list:
    #     ep_name_text_list.append(item.text)

    # all_info_list = zip(ep_href_list, ep_name_text_list)
    
    for ep_link in ep_href_list:
        driver2.get(ep_link)
        
        try:
            ep_name = driver2.find_element(By.XPATH, "//div[@class='sc-80d4314-1 fbQftq']/h1").text
        except: ep_name = None

        try:
            ep_number = driver2.find_element(By.XPATH, "//div[@class='sc-232116ef-0 LumGv']").text
            ep_season = int(ep_number[1:].split('.', 1)[0])
            ep_ep = int(ep_number.split('E')[1])
        except: ep_number, ep_season, ep_ep = None
            
        try:
            ep_date = driver2.find_element(By.XPATH, "//ul[@class='ipc-inline-list ipc-inline-list--show-dividers sc-8c396aa2-0 kqWovI baseAlt']/li[1]").text
            ep_date = datetime.strptime(ep_date.split('d ')[1], '%b %d, %Y').date()
        except: ep_date = None

        try:
            ep_rating = driver2.find_element(By.XPATH, "//span[@class='sc-7ab21ed2-1 jGRxWM']").text
            ep_rating = int(float(ep_rating)*10)
        except: ep_rating = None

        try:
            ep_review_amount = driver2.find_element(By.XPATH, "//div[@class='sc-7ab21ed2-3 dPVcnq']").text
            ep_review_amount = value_to_int(ep_review_amount)
        except: ep_review_amount = None
        
        all_info_list = [ep_link, ep_season, ep_ep, ep_name, ep_date, ep_rating, ep_review_amount]

        with open(csvfile, "a") as f:
            writer = csv.writer(f, delimiter=',' )
            writer.writerow(all_info_list)

driver.close()
driver2.close()
